In [18]:
# Издеваемся над датасетом
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.model_selection import train_test_split

# Загрузка данных
df = pd.read_csv('/content/author_ru_utf.csv')

# Предположим, что колонка 'Author' — это целевая переменная
labels = df['Author']

# Удаляем ненужные колонки, например 'id', 'document'
X = df.drop(['id', 'document', 'Author'], axis=1)
y = labels

# Обработка категориальных переменных
# В вашем случае, все переменные — числовые, кроме 'Author'
# Поэтому, если есть категориальные, делаем one-hot encoding
# Например, если есть текстовые колонки, их нужно преобразовать
# Если все числовые - пропускаем этот шаг

# Для демонстрации, предположим, что все колонки, кроме 'Author', числовые
# Приведем их к диапазону 0-1 (нормализация)
X = X.astype(float)
X = (X - X.min()) / (X.max() - X.min())

from sklearn.preprocessing import LabelEncoder, OneHotEncoder

# Кодируем метки в числа
le = LabelEncoder()
y_encoded = le.fit_transform(y)

# Преобразуем в one-hot
encoder = OneHotEncoder(sparse_output=False)
y_onehot = encoder.fit_transform(y_encoded.reshape(-1, 1))

# Разделение на train, validation и test
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y_onehot, test_size=0.2, random_state=42)
X_train, X_valid, y_train, y_valid = train_test_split(X_train_full, y_train_full, test_size=0.2, random_state=42)

In [35]:
print(X.dtypes)

на        float64
что       float64
за        float64
и         float64
в         float64
из        float64
со        float64
а         float64
во        float64
для       float64
с         float64
но        float64
к         float64
по        float64
от        float64
под       float64
до        float64
про       float64
о         float64
ко        float64
над       float64
без       float64
при       float64
об        float64
через     float64
из-за     float64
из-под    float64
dtype: object


In [21]:
df_train = pd.concat([pd.DataFrame(X_train), pd.DataFrame(y_train, columns=le.classes_)], axis=1)
df_train.to_csv('train_dataset.csv', index=False)

# Аналогично для валидной и тестовой
df_valid = pd.concat([pd.DataFrame(X_valid), pd.DataFrame(y_valid, columns=le.classes_)], axis=1)
df_valid.to_csv('valid_dataset.csv', index=False)

df_test = pd.concat([pd.DataFrame(X_test), pd.DataFrame(y_test, columns=le.classes_)], axis=1)
df_test.to_csv('test_dataset.csv', index=False)

In [37]:
print(X.dtypes)

на        float64
что       float64
за        float64
и         float64
в         float64
из        float64
со        float64
а         float64
во        float64
для       float64
с         float64
но        float64
к         float64
по        float64
от        float64
под       float64
до        float64
про       float64
о         float64
ко        float64
над       float64
без       float64
при       float64
об        float64
через     float64
из-за     float64
из-под    float64
dtype: object


In [41]:
from keras.models import Sequential
from keras.layers import Dense, Activation

import numpy as np
import re

In [54]:
def get_model():
    # function returns the many-layered feedforward network
    # considers 9 floating point numbers as input
    # produces 2 floating point numbers as output
    model = Sequential()
    model.add(Dense(18, activation='relu', input_dim=9))
    # uncomment the following line to add one more layer
    # model.add(Dense(9, activation='relu'))
    model.add(Dense(2, activation='softmax'))
    model.compile(optimizer='adam',
                  loss='mean_squared_error',
                  metrics=['accuracy'])
    return model
def prepare_data(filename):
    with open(filename) as f:
        lines = f.readlines()

    # Создаем списки для хранения данных
    input_list = []
    output_list = []

    # Проходим по всем строкам файла, пропуская заголовки (если есть)
    for i in range(len(lines)):
        line = lines[i].rstrip()
        # Удаляем все ненужные символы, оставляя только числа
        line = re.sub(r'[^\d\.\-e]', ' ', line)
        line_data = line.split()
        line_data = [x for x in line_data if x.strip() != '']

        # Проверка, что данных достаточно (минимум 11 чисел)
        if len(line_data) >= 11:
            # Преобразуем в float и берем нужные части
            values = list(map(float, line_data))
            # Первые 9 — входные данные
            input_list.append(values[0:9])
            # Следующие 2 — выходные данные
            output_list.append(values[9:11])
        else:
            print(f"Строка {i+1} содержит недостаточно данных: {len(line_data)} элементов.")

    # Преобразуем списки в numpy массивы
    input_data = np.array(input_list)
    output_data = np.array(output_list)

    # Теперь делим данные на обучающие, валидационные и тестовые
    total_samples = len(input_data)
    train_end = int(0.7 * total_samples)
    valid_end = int(0.85 * total_samples)

    train_data = input_data[:train_end]
    train_labels = output_data[:train_end]

    valid_data = input_data[train_end:valid_end]
    valid_labels = output_data[train_end:valid_end]

    test_data = input_data[valid_end:]
    test_labels = output_data[valid_end:]

    return train_data, train_labels, valid_data, valid_labels, test_data, test_labels



In [55]:
# getting data for both training and testing
train_data, train_labels, valid_data, valid_labels, test_data, test_labels = prepare_data('/content/test_dataset.csv')

# Создаем модель
model = get_model()

# Обучаем модель
model.fit(train_data, train_labels, validation_data=(valid_data, valid_labels),
          epochs=100, batch_size=32)

# Предсказания модели
predictions = model.predict_on_batch(test_data)

# Подсчет процента правильных предсказаний
correct = 0
for i in range(test_data.shape[0]):
    true_class = 0 if test_labels[i, 0] > test_labels[i, 1] else 1
    pred_class = 0 if predictions[i, 0] > predictions[i, 1] else 1
    if true_class == pred_class:
        correct += 1

accuracy = correct / test_data.shape[0]
print('-------------------')
print(f"Процент правильных предсказаний: {accuracy * 100:.2f}%")
print('-------------------')


Строка 1 содержит недостаточно данных: 2 элементов.
Строка 94 содержит недостаточно данных: 4 элементов.
Строка 95 содержит недостаточно данных: 4 элементов.
Строка 96 содержит недостаточно данных: 4 элементов.
Строка 97 содержит недостаточно данных: 4 элементов.
Строка 98 содержит недостаточно данных: 4 элементов.
Строка 99 содержит недостаточно данных: 4 элементов.
Строка 100 содержит недостаточно данных: 4 элементов.
Строка 101 содержит недостаточно данных: 4 элементов.
Строка 102 содержит недостаточно данных: 4 элементов.
Строка 103 содержит недостаточно данных: 4 элементов.
Строка 104 содержит недостаточно данных: 4 элементов.
Строка 105 содержит недостаточно данных: 4 элементов.
Строка 106 содержит недостаточно данных: 4 элементов.
Строка 107 содержит недостаточно данных: 4 элементов.
Строка 108 содержит недостаточно данных: 4 элементов.
Строка 109 содержит недостаточно данных: 4 элементов.
Строка 110 содержит недостаточно данных: 4 элементов.
Строка 111 содержит недостаточно дан

/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 1s 227ms/step - accuracy: 0.7292 - loss: 0.0589 - val_accuracy: 0.8571 - val_loss: 0.0455
Epoch 2/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.8958 - loss: 0.0581 - val_accuracy: 0.8571 - val_loss: 0.0451
Epoch 3/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.9167 - loss: 0.0559 - val_accuracy: 0.8571 - val_loss: 0.0448
Epoch 4/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.9583 - loss: 0.0580 - val_accuracy: 0.8571 - val_loss: 0.0447
Epoch 5/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.9479 - loss: 0.0556 - val_accuracy: 0.8571 - val_loss: 0.0447
Epoch 6/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.9479 - loss: 0.0536 - val_accuracy: 0.8571 - val_loss: 0.0448
Epoch 7/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.9479 - loss: 0.0523 - val_accuracy: 0.8571 - val_loss: 0.0450
Epoch 8/100
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.9375 - loss: 0.0535 - val_accuracy: 0.8571 - val_loss